https://wikidocs.net/256983

In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [7]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Structured-Output-Chain")

LangSmith 추적을 시작합니다.
[프로젝트명]
Structured-Output-Chain


In [30]:
from langchain.chains.openai_functions import create_structured_output_runnable
# from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Quiz(BaseModel):
    """4지선다형 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 4개의 선택지 입니다.")


llm = ChatUpstage(model="solar-pro", temperature=0.8)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 4지선다형 퀴즈를 한국어로 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 여러 개의 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "수능 지구과학2"})

# 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print()

1. 지구과학2 시험에 출제된 문제 중, 다음 중 중력에 관련된 이론을 제시한 인물은 누구인가? (난이도: 보통)
   1) 갈릴레오 갈릴레이
   2) 윌리엄 핼리
   3) 찰스 다윈
   4) 벤자민 프랭클린

2. 지구과학2에서 다루는 대륙의 이동설에 대한 설명으로 옳지 않은 것은? (난이도: 어려움)
   1) 대륙이 뜨고 내린다.
   2) 지각은 하나의 판으로 이루어져 있다.
   3) 대륙은 거대한 하나의 땅덩어리였다.
   4) 지각은 여러 개의 판으로 이루어져 있다.

3. 지구과학2에서 다루는 판 구조론에 대한 설명으로 옳지 않은 것은? (난이도: 어려움)
   1) 판은 서로 밀고 당긴다.
   2) 판의 경계는 서로 밀고 당긴다.
   3) 판의 경계는 서로 밀고 당긴다.
   4) 판의 경계는 서로 밀고 당긴다.



In [31]:
from langchain.chains.openai_functions import create_structured_output_runnable
# from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Quiz(BaseModel):
    """OX 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 답은 O 또는 X 입니다.")


llm = ChatUpstage(model="solar-pro", temperature=0.8)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 OX 퀴즈를 한국어로 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 여러 개의 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "수능 지구과학2"})

# 출력
for idx, quiz in enumerate(quiz.quizzes, 1):
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print()

1. 지구는 평평한 판으로 이루어져 있다. (난이도: Easy)
   1) X
   2) O

2. 달은 자체적인 빛을 내뿜는다. (난이도: Easy)
   1) X
   2) O



In [34]:
quiz

Quiz(question='달은 자체적인 빛을 내뿜는다.', level='Easy', options=['X', 'O'])

In [42]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Quiz(BaseModel):
    """OX 퀴즈의 정보를 추출합니다"""
    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)")
    options: List[str] = Field(..., description="퀴즈의 답은 O 또는 X 입니다.")
    answer: str = Field(..., description="정답은 O 또는 X 입니다.")  # 정답 추가

llm = ChatUpstage(model="solar-pro", temperature=0.8)

# 텍스트를 분석하여 관련 개념에 대한 OX 퀴즈를 생성하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a quiz master who generates OX quizzes based on the provided context. Ensure the quizzes are scientifically accurate and in OX format.",
        ),
        (
            "human",
            "제공된 내용에 대해 OX 퀴즈를 5개 만들어 주세요. 텍스트 내용에서 잘못된 부분이 있으면 그것도 반영하여 출제해주세요. \n\nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format."),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 주어진 내용에 대해 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작. 중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. 원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승."})

# 생성된 퀴즈 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print(f"정답: {quiz.answer}")  # 정답 출력
    print()


1. 별의 형성에서 성운 단계는 무엇인가요? (난이도: 보통)
   1) 질량 증가
   2) 온도 하강
   3) 수축
정답: 수축

2. 가스와 먼지 구름에서 수축하는 단계는 무엇인가요? (난이도: 보통)
   1) 온도 상승
   2) 밀도 증가
   3) 수축
정답: 수축

3. 원시별 단계에서 어떤 일이 일어나나요? (난이도: 보통)
   1) 핵융합 반응
   2) 수축 과정
   3) 질량 증가
정답: 핵융합 반응

4. 원시별 단계에서 중심부 온도는 얼마인가요? (난이도: 보통)
   1) 1억 K
   2) 1천만 K
   3) 1만 K
정답: 1천만 K

5. 성운 단계에서 중심부가 차갑게 되는 단계는 무엇인가요? (난이도: 보통)
   1) 온도 상승
   2) 밀도 증가
   3) 수축
정답: 수축



In [40]:
generated_quizzes

QuizList(quizzes=[Quiz(question='별의 형성 성운 단계에서 성운은 가스와 먼지 구름의 수축으로 인해 온도가 상승하는가?', level='보통', options=['O', 'X']), Quiz(question='초기 원시별 단계에서 중심부 온도는 1천만 K로 상승하는가?', level='보통', options=['O', 'X'])])